In [67]:
import os
import sys
if not os.getcwd().endswith("Submodular"):
    sys.path.append('../../Submodular')    

In [68]:
import DeviceDir

DIR, RESULTS_DIR = DeviceDir.get_directory()
device, NUM_PROCESSORS = DeviceDir.get_device()

In [69]:
from ipynb.fs.full.Dataset import get_data
from ipynb.fs.full.Dataset import datasets as available_datasets
from ipynb.fs.full.Utils import save_plot

In [70]:
import argparse
import sys
import os
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_undirected, sort_edge_index
from torch_geometric.data import NeighborSampler, ClusterData, ClusterLoader, Data, GraphSAINTNodeSampler, GraphSAINTEdgeSampler, GraphSAINTRandomWalkSampler, RandomNodeSampler
from torch_scatter import scatter

from logger import Logger, SimpleLogger
from dataset import load_nc_dataset, NCDataset
from data_utils import normalize, gen_normalized_adjs, evaluate, eval_acc, eval_rocauc, to_sparse_tensor
from parse import parse_method, parser_add_main_args
from batch_utils import nc_dataset_to_torch_geo, torch_geo_to_nc_dataset, AdjRowLoader, make_loader

In [71]:
import argparse
from argparse import ArgumentParser

#set default arguments here
def get_configuration():
    
    parser = ArgumentParser()
    
    ### Parse args ###
    parser = argparse.ArgumentParser(description='General Training Pipeline')
    parser_add_main_args(parser)
    parser.add_argument('--train_batch', type=str, default='cluster', help='type of mini batch loading scheme for training GNN')
    parser.add_argument('--no_mini_batch_test', action='store_true', help='whether to test on mini batches as well')
    parser.add_argument('--batch_size', type=int, default=10000)
    parser.add_argument('--num_parts', type=int, default=100, help='number of partitions for partition batching')
    parser.add_argument('--cluster_batch_size', type=int, default=1, help='number of clusters to use per cluster-gcn step')
    parser.add_argument('--saint_num_steps', type=int, default=5, help='number of steps for graphsaint')
    parser.add_argument('--test_num_parts', type=int, default=10, help='number of partitions for testing')
    
    #parser.add_argument('--epochs', type=int, default=1)
    parser.add_argument('--log_info', type=bool, default=True)
    parser.add_argument('--pbar', type=bool, default=False)
    #parser.add_argument('--batch_size', type=int, default=2048)
    parser.add_argument('--learning_rate', type=float, default=0.01)
    parser.add_argument('--num_gpus', type=int, default=-1)
    parser.add_argument('--parallel_mode', type=str, default="dp", choices=['dp', 'ddp', 'ddp2'])
    #parser.add_argument('--dataset', type=str, default="Cora", choices=available_datasets)
    #parser.add_argument('--use_normalization', action='store_false', default=True)
    parser.add_argument('--use_normalization', action='store_true')    
    parser.add_argument('-f') ##dummy for jupyternotebook
    
    args = parser.parse_args()
    
    dict_args = vars(args)
    
    return args, dict_args

args, dict_args = get_configuration()

In [72]:
import os.path as osp
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch_geometric.datasets import LINKXDataset
# from torch_geometric.nn import LINKX
import numpy as np
from tqdm import tqdm
from torch_geometric.loader import NeighborSampler, NeighborLoader
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn import GCNConv, SGConv, GATConv, JumpingKnowledge, APPNP, GCN2Conv, MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
import scipy.sparse

# LINKX model

### Available models
LINK, GCN, MLP, SGC, GAT, SGCMem, MultiLP, MixHop, 

GCNJK, GATJK, H2GCN, APPNP_Net, LINK_Concat, LINKX, GPRGNN, GCNII

### Available Sampler

NeighborSampler, 

ClusterData, ClusterLoader, 

GraphSAINTNodeSampler, GraphSAINTEdgeSampler, GraphSAINTRandomWalkSampler, 

RandomNodeSampler

# Train

In [73]:
def test(model, loader, mask, name='Train'):
    
    if args.log_info:
        pbar = tqdm(total=sum(mask).item())
        pbar.set_description(f'Evaluating {name}')

    model.eval()
    
    total_correct=0
    total_examples=0
    
    with torch.no_grad():                  
    
        for i,batch_data in enumerate(loader):
            batch_data = batch_data.to(device)
            batch_dataset = torch_geo_to_nc_dataset(batch_data, device=device)
            out = model(batch_dataset)
            out=out[:batch_data.batch_size,:]
            pred = out.argmax(dim=-1)            
            correct = pred.eq(batch_data.y[:batch_data.batch_size].to(device))

            total_correct+=correct.sum()
            total_examples+=batch_data.batch_size
            
            if args.log_info:
                pbar.update(batch_data.batch_size)
    
    if args.log_info:
        pbar.close()

    return total_correct.item()/total_examples

In [74]:
def train(model, data, epochs, train_neighbors=[8,4], test_neighbors=[8,4]):
    
    if args.log_info:
        print("Train Neighbors: ", train_neighbors)
        print("Test Neighbors: ", test_neighbors)
     
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-3)    
    batch_size=1024
    loader = NeighborLoader(data, input_nodes=data.train_mask,num_neighbors=train_neighbors, 
                            batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = NeighborLoader(data,input_nodes=data.val_mask, num_neighbors=test_neighbors, 
                                batch_size=batch_size,shuffle=False, num_workers=0)
    test_loader = NeighborLoader(data, input_nodes=data.test_mask,num_neighbors=test_neighbors, 
                                 batch_size=batch_size,shuffle=False, num_workers=0)    
    
    train_losses=[]
    best_acc = 0 
    num_iteration = epochs
    
    for epoch in range(1,epochs+1):
        
        if args.log_info:
            pbar = tqdm(total=int(sum(data.train_mask)))
            pbar.set_description(f'Epoch {epoch:02d}')
        
        model.train()
        total_loss = total_examples = 0
        
        for i,batch_data in enumerate(loader):                
            batch_data = batch_data.to(device)
            batch_dataset = torch_geo_to_nc_dataset(batch_data, device=device)
            
            optimizer.zero_grad()            
            out = model(batch_dataset)
            
            #loss = F.nll_loss(out[batch_data.train_mask], batch_data.y[batch_data.train_mask])
            loss = F.cross_entropy(out[batch_data.train_mask], batch_data.y[batch_data.train_mask])

            loss.backward()
            optimizer.step()
            total_loss += loss.item() * sum(batch_data.train_mask).item()
            total_examples += sum(batch_data.train_mask).item()
            
            if args.log_info:
                pbar.update(batch_size)
        
        if args.log_info:
            pbar.close()
        
        loss=total_loss / total_examples
        train_losses.append(loss)     
        
        #train_acc=test(model, train_loader,data.train_mask,'Train')
        train_acc=0
        val_acc = test(model, val_loader, data.val_mask,'Validation')
        test_acc = test(model, test_loader, data.test_mask,'Test')
                
        if test_acc>best_acc:
            best_acc=test_acc
        
        std_dev = np.std(train_losses[-5:])
        
        if args.log_info:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}, Std dev: {std_dev:.4f}')
                
        if epoch>=5 and std_dev<=1e-4:
            num_iteration = epoch
            
            if args.log_info:                
                print("Iteration for convergence: ", epoch)
            break
                
    return best_acc, num_iteration

In [75]:
def LINKXperformanceSampler(data, dataset, num_classes, epochs=1, train_neighbors=[8,4], test_neighbors=[8,4]):        
    from models import LINKX
    
    model = LINKX(in_channels=data.x.shape[1], hidden_channels=64, out_channels = dataset.num_classes, 
                        num_layers=1, num_nodes=data.num_nodes).to(device)
    if args.log_info:
        print(model) 
    
    best_acc, num_iteration = train(model, data, epochs, train_neighbors=train_neighbors, test_neighbors=test_neighbors)    
    
    return best_acc, num_iteration, model

In [80]:
args.log_info = True
DATASET_NAME = 'karate'
data, dataset = get_data(DATASET_NAME, DIR=None, log=False, h_score=True, split_no=0); print("")
print(data)
best_acc, num_iteration, _ = LINKXperformanceSampler(data, dataset, dataset.num_classes, epochs=100, train_neighbors=[8,4], test_neighbors=[8,4])
print(best_acc, num_iteration)

N  34  E  156  d  4.588235294117647 0.8020520210266113 0.7564102411270142 0.6170591711997986 -0.4756128787994385 
Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34], val_mask=[34], test_mask=[34])
LINKX(
  (mlpA): MLP(
    (lins): ModuleList(
      (0): Linear(in_features=34, out_features=64, bias=True)
    )
    (bns): ModuleList()
  )
  (mlpX): MLP(
    (lins): ModuleList(
      (0): Linear(in_features=34, out_features=64, bias=True)
    )
    (bns): ModuleList()
  )
  (W): Linear(in_features=128, out_features=64, bias=True)
  (mlp_final): MLP(
    (lins): ModuleList(
      (0): Linear(in_features=64, out_features=4, bias=True)
    )
    (bns): ModuleList()
  )
)
Train Neighbors:  [8, 4]
Test Neighbors:  [8, 4]


Epoch 01: : 1024it [00:00, 160679.66it/s]      
Evaluating Test: 100%|██████████| 30/30 [00:00<00:00, 15791.81it/s]


Epoch: 001, Train Loss: 1.4216, Train: 0.0000, Val: 0.3000, Test: 0.3333, Std dev: 0.0000


Epoch 02: : 1024it [00:00, 257507.48it/s]      
Evaluating Test: 100%|██████████| 30/30 [00:00<00:00, 14359.14it/s]


Epoch: 002, Train Loss: 1.3428, Train: 0.0000, Val: 0.3667, Test: 0.3333, Std dev: 0.0394


...

Epoch: 028, Train Loss: 0.0001, Train: 0.0000, Val: 0.2333, Test: 0.2000, Std dev: 0.0003


Epoch 29: : 1024it [00:00, 264908.86it/s]      
Evaluating Test: 100%|██████████| 30/30 [00:00<00:00, 15395.71it/s]


Epoch: 029, Train Loss: 0.0001, Train: 0.0000, Val: 0.2000, Test: 0.2333, Std dev: 0.0002


Epoch 30: : 1024it [00:00, 256293.55it/s]      
Evaluating Test: 100%|██████████| 30/30 [00:00<00:00, 15501.92it/s]

Epoch: 030, Train Loss: 0.0001, Train: 0.0000, Val: 0.2000, Test: 0.2333, Std dev: 0.0001
Iteration for convergence:  30
0.5666666666666667 30


In [77]:
# import time
# from torch_geometric.loader import ClusterData, ClusterLoader, NeighborSampler

In [78]:
# sampler_dir = DIR+'ClusterGCNtest/'+DATASET_NAME
# if not os.path.exists(sampler_dir):
#     os.makedirs(sampler_dir)

# num_parts=2

# start_time = time.time()
# cluster_data = ClusterData(data, num_parts=num_parts, recursive=False,save_dir=sampler_dir)
# train_loader = ClusterLoader(cluster_data, batch_size=1, shuffle=True,num_workers=0)
# subgraph_loader = NeighborSampler(data.edge_index, sizes=[-1], batch_size=1024,shuffle=False, num_workers=0)
# end_time = time.time()

# print(cluster_data)

# Batch Experiments

In [79]:
def batch_experiments(num_run=1):
    
    ALL_DATASETs= [
#         "Roman-empire","Texas","Squirrel","Chameleon",
#         "Cornell","Actor","Wisconsin","Flickr","Amazon-ratings","reed98","amherst41","genius",
        "AmazonProducts",
#         "cornell5","penn94","johnshopkins55",
        "Yelp",
#         "cora","Tolokers","Minesweeper",
#         "CiteSeer","Computers","PubMed","pubmed",
        "Reddit",
#         "cora_ml","dblp",
        "Reddit2",
#         "Cora","CS","Photo","Questions","Physics","citeseer",                
    ]
 
    
#     ALL_DATASETs= ["karate"]
    
    args.log_info = False
    
    for DATASET_NAME in ALL_DATASETs:  
        print(DATASET_NAME, end=' ')
        
        result_file = open("Results/LINKX.txt",'a+')        
        result_file.write(f'{DATASET_NAME} ')
                
        accs = []
        itrs = []
        
        for i in range(num_run):
            data, dataset = get_data(DATASET_NAME, DIR=None, log=False, h_score=False, split_no=i)   
            
#             if data.num_nodes>100000:
#                 accs.append(-1)
#                 itrs.append(-1)
#                 break
            
            if len(data.y.shape) > 1:
                data.y = data.y.argmax(dim=1)        
                num_classes = torch.max(data.y).item()+1
            else:
                num_classes = dataset.num_classes
            
            if num_classes!= torch.max(data.y)+1:
                num_classes = torch.max(data.y).item()+1
                
            if data.num_nodes<100000:
                max_epochs = 150
            else:
                max_epochs = 50
                              
            accuracy, itr, _ = LINKXperformanceSampler(data, dataset, num_classes, epochs=max_epochs, train_neighbors=[8,4], test_neighbors=[8,4])
            
            accs.append(accuracy)
            itrs.append(itr)
            #print(itr, accuracy)
                        
        #print(accs, itrs)
        print(f'acc {np.mean(accs):0.4f} sd {np.std(accs):0.4f} itr {int(np.mean(itrs)):d} sd {int(np.std(itrs)):d}')
        result_file.write(f'acc {np.mean(accs):0.4f} sd {np.std(accs):0.4f} itr {int(np.mean(itrs)):d} sd {int(np.std(itrs)):d}\n')
        result_file.close()
                
# batch_experiments(num_run=5)